# Import libraries and environment keys

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

# Orchestration Implementation
We can unify the method to call any model from SAP GenAI Hub using the Orchestration Service.

In [11]:
from gen_ai_hub.orchestration.utils import load_text_file
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, ProfileEntity
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.service import OrchestrationService


data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)
config = OrchestrationConfig(
    template=Template(
        messages=[
            SystemMessage("You are a helpful AI assistant."),
            UserMessage("Summarize the following CV in 10 sentences: {{?orgCV}}"),
        ]
    ),
    llm=LLM(
        name="gpt-4o",
    ),
    data_masking=data_masking
)
cv_as_string = "" \
"Name: John Doe\
Address: 1234 Elm St, Springfield, IL, 62701 \
Phone Number: (555) 123-4567 \
Email: john.doe@example.com \
Credit Card Number: 4111 1111 1111 1111\
Date of Birth: 01/01/1980 \
Social Security Number: 123-45-6789"

orchestration_service = OrchestrationService(config=config)
result = orchestration_service.run(
    config=config,
    template_values=[
        TemplateValue(name="orgCV", value=cv_as_string)
    ]
)

In [12]:
print(result.orchestration_result.choices[0].message.content)

The CV belongs to MASKED_PERSON, who resides at 1234 Elm St, MASKED_LOCATION, MASKED_LOCATION. They are reachable via phone at (555) 123-4567 and email at MASKED_EMAIL. MASKED_PERSON was born on January 1, 1980. Personal sensitive information such as credit card and MASKED_ORG numbers appear to be included in the CV, with the credit card number being 4111 1111 1111 1111 and the MASKED_ORG number listed as 123-45-6789. This document likely contains more detailed information regarding MASKED_PERSON's professional experiences, educational background, and skills, although these specifics are not visible in the summary provided. Typically, a CV highlights achievements, qualifications, and work history relevant to job applications. Contact information listed indicates how employers can reach or verify the candidate's credentials. It is essential to ensure sensitive information is properly redacted before sharing such a document publicly to protect privacy.
